In [8]:
import sqlite3
from datetime import datetime

today_str = datetime.today().strftime('%Y%m%d')

max_date = ''
with sqlite3.connect('faq20072.db') as conn:
    cursor = conn.cursor()
    cursor.execute('SELECT MAX(faq.regDate) max_date FROM faq')
    result = cursor.fetchone()
    max_date = result[0]
    
print(f'최종 입수 일시 : {max_date} / 일자 {max_date[:8]} ~ 오늘 {today_str}')

최종 입수 일시 : 20250512224548 / 일자 20250512 ~ 오늘 20250514


In [9]:
import requests
import os

# 민원정책 질의응답조회서비스
url = 'http://apis.data.go.kr/1140100/CivilPolicyQnaService/PolicyQnaList'
d_key = os.getenv('SINNUNGO_DECODING_KEY', 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789')

params ={
    'serviceKey' : d_key, 
    'firstIndex' : '1', 
    'recordCountPerPage' : '30000', 
    'regFrom' : max_date[:8], 
    'regTo' : today_str
}

response = requests.get(url, params=params)

r = response.json()
faq_list = r['resultList']
print(f"입수 {len(faq_list)} 건의 데이터")

입수 193 건의 데이터


In [10]:
with sqlite3.connect('faq20072.db') as conn:
    cursor = conn.cursor()
    
    # 테이블 삭제
    cursor.execute('DROP TABLE IF EXISTS faq_new')
    
    # 테이블 생성
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS faq_new (
        faqNo TEXT PRIMARY KEY,
        dutySctnNm TEXT,
        ancCode TEXT,
        ancName TEXT,
        regDate TEXT,
        title TEXT
    )
    ''')
    
    success_count = 0
    for item in faq_list:
        try:
            cursor.execute('''
                INSERT INTO faq_new (faqNo, dutySctnNm, ancCode, ancName, regDate, title)
                VALUES (?, ?, ?, ?, ?, ?)
            ''', (
                item['faqNo'],
                item['dutySctnNm'],
                item['ancCode'],
                item['ancName'],
                item['regDate'],
                item['title']
            ))
            success_count = success_count + 1
        except sqlite3.IntegrityError:
            print(f"Duplicate entry for faqNo: {item['faqNo']}")
            
    print(f"입수 {success_count} 건의 데이터")

입수 193 건의 데이터


In [11]:
import sqlite3

with sqlite3.connect('faq20072.db') as conn:
    cursor = conn.cursor()
    
     # 1. INSERT 전 row 수 확인
    cursor.execute('SELECT COUNT(*) FROM faq')
    before_count = cursor.fetchone()[0]
    
     # 2. INSERT 실행 (faq_new에만 있는 행을 추가)
    cursor.execute('''
        INSERT INTO faq
        SELECT n.* 
        FROM faq_new n
        LEFT JOIN faq f ON n.faqNo = f.faqNo
        WHERE f.faqNo IS NULL
        ''')
    conn.commit()
    
     # 3. INSERT 후 row 수 확인
    cursor.execute('SELECT COUNT(*) FROM faq')
    after_count = cursor.fetchone()[0]
    
    inserted_count = after_count - before_count
    print(f'Inserted rows: {inserted_count} ( {before_count} -> {after_count})')


Inserted rows: 41 ( 20463 -> 20504)


In [12]:
with sqlite3.connect('faq20072.db') as conn:
    cursor = conn.cursor()
    
    # faq 테이블에서 필요한 항목 조회
    # cursor.execute('SELECT faqNo, dutySctnNm FROM faq ORDER BY faqNo')
    # LEFT JOIN을 사용하여 faq_de에 없는 항목만 가져오기
    cursor.execute('''SELECT f.faqNo, f.dutySctnNm 
    FROM faq f
    LEFT JOIN faq_de d ON f.faqNo = d.faqNo
    WHERE d.faqNo IS NULL''')

    faq_news = cursor.fetchall()
    print(f"새로운 입수 대상: {len(faq_news)}")

새로운 입수 대상: 73


In [13]:
import sqlite3
import requests
import time  # 요청 간 텀 주기 위해

# 인증키 및 API 기본 정보
API_URL = 'http://apis.data.go.kr/1140100/CivilPolicyQnaService/PolicyQnaItem'
API_KEY = d_key 

with sqlite3.connect('faq20072.db') as conn:
    cursor = conn.cursor()
    
    # 상세 정보 저장용 테이블 생성
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS faq_de (
        faqNo TEXT PRIMARY KEY,
        qnaTitl TEXT,
        qstnCntnCl TEXT,
        ansCntnCl TEXT,
        ancName TEXT,
        deptName TEXT,
        regDate TEXT,
        ancCode TEXT,
        deptCode TEXT
    )
    ''')
    
    
    ok_count = 0
    fail_count = 0

    # 루프 돌며 상세 정보 가져와 DB 저장
    for faqNo, dutySctnNm in faq_news:
        params = {
            'serviceKey': API_KEY,
            'faqNo': faqNo,
            'dutySctnNm': dutySctnNm
        }

        try:
            response = requests.get(API_URL, params=params)
            data = response.json()
            if data['resultCode'] != 'S00':
                print(f"[{faqNo}] API 오류: {data['resultMessage']}")
                continue

            item = data['resultData']
            cursor.execute('''
                INSERT OR REPLACE INTO faq_de (
                    faqNo, qnaTitl, qstnCntnCl, ansCntnCl,
                    ancName, deptName, regDate, ancCode, deptCode
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                item.get('faqNo'),
                item.get('qnaTitl'),
                item.get('qstnCntnCl'),
                item.get('ansCntnCl'),
                item.get('ancName'),
                item.get('deptName'),
                item.get('regDate'),
                item.get('ancCode'),
                item.get('deptCode')
            ))
            # print(f"[{faqNo}] 저장 완료")
            # time.sleep(0.3)  # 과도한 요청 방지
            ok_count = ok_count + 1
            if ok_count % 100 == 0:
                print(f"[{faqNo}] {ok_count}개 저장됨")
                conn.commit()
                time.sleep(0.3)  # 과도한 요청 방지

        except Exception as e:
            fail_count = fail_count + 1
            print(f"[{faqNo}] 처리 중 예외 발생: {e}", end=' ')
    conn.commit()  # 마지막 커밋
    print(f"총 { len(faq_news) } - {ok_count}건 저장 완료, 실패 {fail_count}건")

[6881095] API 오류: 조회된 데이터가 없습니다.
[6881433] API 오류: 조회된 데이터가 없습니다.
[6881466] API 오류: 조회된 데이터가 없습니다.
[6880775] API 오류: 조회된 데이터가 없습니다.
[6880235] API 오류: 조회된 데이터가 없습니다.
[6873621] API 오류: 조회된 데이터가 없습니다.
[6867062] API 오류: 조회된 데이터가 없습니다.
[6876947] API 오류: 조회된 데이터가 없습니다.
[6876454] API 오류: 조회된 데이터가 없습니다.
[6875441] API 오류: 조회된 데이터가 없습니다.
[6870886] API 오류: 조회된 데이터가 없습니다.
[6872406] API 오류: 조회된 데이터가 없습니다.
[6872402] API 오류: 조회된 데이터가 없습니다.
[6873504] API 오류: 조회된 데이터가 없습니다.
[6873483] API 오류: 조회된 데이터가 없습니다.
[6872935] API 오류: 조회된 데이터가 없습니다.
[6872628] 처리 중 예외 발생: Expecting value: line 1 column 1 (char 0) [6870963] API 오류: 조회된 데이터가 없습니다.
[6870962] API 오류: 조회된 데이터가 없습니다.
[6871466] API 오류: 조회된 데이터가 없습니다.
[6869745] 처리 중 예외 발생: Expecting value: line 1 column 1 (char 0) [6869007] API 오류: 조회된 데이터가 없습니다.
[6868677] API 오류: 조회된 데이터가 없습니다.
[6868665] API 오류: 조회된 데이터가 없습니다.
[6868672] API 오류: 조회된 데이터가 없습니다.
[6868667] API 오류: 조회된 데이터가 없습니다.
[6868662] API 오류: 조회된 데이터가 없습니다.
[6868783] API 오류: 조회된 데이터가 없습니다.
[6866333] API 